<a href="https://colab.research.google.com/github/satomko/pdfchatbot/blob/main/pdfChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install langchain langchain_community openai faiss-cpu tiktoken
import os


In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files

# Upload the PDF file
uploaded = files.upload()

# Get the name of the uploaded file
file_name = list(uploaded.keys())[0]

# Load the PDF
loader = PyPDFLoader(file_name)
pages = loader.load_and_split()

# Chunk it up
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=24)
chunks = text_splitter.split_documents(pages)

Saving Scott_Tomko_Resume.pdf to Scott_Tomko_Resume (1).pdf


In [21]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

In [16]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    db.as_retriever(),
    return_source_documents=True
)

chat_history = []

def chat(query):
    result = qa_chain({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    return result['answer']

/tmp/ipython-input-2458226057.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(),


In [22]:
print(chat("What's the main topic of this PDF?"))
print(chat("Can you summarize the key points?"))

The achievements and leadership impact of the individual mentioned in the PDF include improving quality across the organization, contributing to positioning Adobe in the product "Leader" quadrant for analytics solutions by Gartner and Forrester, directing over $6M ACV in software development projects at the Pension Benefit Guaranty Corporation (PBGC) with a team of 46 individuals, managing software development engagements at the National Institutes of Health (NIH), and possessing skills in Problem Solving and Critical Thinking, Stakeholder Relationship Management, Resource Planning, Budget Planning, Talent Development, Performance Management, Cross-functional Leadership, Incident Management, OKRs, and KPIs.
The key achievements and leadership impact of the individual mentioned in the PDF include skills such as Solving and Critical Thinking, Stakeholder Relationship Management, Resource Planning, Budget Planning, Talent Development, Performance Management, Cross-functional Leadership, I

In [23]:
print(chat("What is the most recent company where the candidate has worked?"))

The candidate most recently worked at Adobe in Lehi, UT.
